## FIRST STEPS IN EXPLORING EEG DATA

Now that we have visualized and explore EEG data using Anywave, we will try to carry out similar work using Python and Python-MNE, a tool used by many researchers in EEG, MEG and fMRI.
Make sure that the folders containing your data are in the same directory as your Jupyter notebooks and Python scripts. This will make it easier to load in data.

In [ ]:
# Magic command to allow us to interact with figures in Jupyter; only works in Jupyter.
import os
import matplotlib.pyplot as plt
import numpy as np
import mne
import ipympl
import pandas as pd

%matplotlib qt

In [ ]:
'''
    Load in an BDF dataset. It should be in the same directory as this script.
    We will read in the bdf file using mne input-output module.
    This will load in a raw object.
    Then plot all channels of the raw data.
'''
%matplotlib widget
fname = 'sub-001_eeg_sub-001_task-think1_eeg.bdf'
rawIn = mne.io.read_raw_bdf(fname, preload=True)

scale_dict = dict(mag=1e-12, grad=4e-11, eeg=20e-6, eog=150e-6, ecg=5e-4,
     emg=1e-3, ref_meg=1e-12, misc=1e-3, stim=1,
     resp=1, chpi=1e-4)
mne.viz.plot_raw(rawIn, duration=5.0, scalings=scale_dict, remove_dc=True)

### The external channels 

Ext1 and Ext2: HEOG; 
Ext3 and Ext 4: VEOG (blinks); 
Ext 7: ECG
Ext5: M1
Ext6: M2
Ext8: FP1

In [ ]:
# Set the auxiliary channels to type 'misc' and the stimulus channel to type 'stim'.
# This allows each data type to be scaled appropriately when visualized.

my_dict={'EXG1': 'eog', 'EXG2': 'eog', 'EXG3': 'eog', 'EXG4': 'eog', 'EXG5': 'eog' , 'EXG6': 'eog', 'EXG7': 'eog', 'EXG8': 'eog',
        'GSR1': 'misc', 'GSR2': 'misc', 'Erg1': 'stim', 'Erg2': 'stim', 'Resp': 'resp', 'Plet': 'misc', 'Temp': 'misc', 'Status': 'stim'}
print(my_dict)
rawIn.set_channel_types(my_dict)

mne.viz.plot_raw(rawIn, duration=5.0, scalings=scale_dict, remove_dc=True)

In [ ]:
# Extract the data from the raw file and get the dimensions of the data.

data1 = rawIn.get_data()     
data_dim = data1.shape
print(data_dim)
print('The number of channels in the current raw dataset is: ', data_dim[0])
print('\nThe number of time samples in the current raw dataset is: ', data_dim[1])

In [ ]:
# Let's take a look at the rawmed1.info attribute, which contains an 'info' object.
# This info object is a python dictionary.

print(rawIn.info)


In [ ]:
# Let's find the following information from the info object.
# - Sampling frequency (Hz)
# - Channel labels (electrode names)


sfreq = rawIn.info['sfreq']
print('The sampling frequency of the current dataset is: ', sfreq)

chan_names = rawIn.info['ch_names']
print(chan_names)

# print out the limit of the lowpass filter applied...


## TASK 1
Now that you know the sampling rate of the current raw dataset, you know how many times per second the EEG signal was sampled.
You also know the number of time samples of the dataset.
With this information, you are going to create the time vector. The time vector shows the time (in seconds) that corresponds to each sample of the data.

CLUE: Given that you know the sampling frequency, what is the time step from one sample to the next?
      Remember that the sampling frequency tells you how many samples there for every second. 

In [ ]:
## -----------------------We know the sampling rate of the data, so can we construct the time vector?----------------------
'''
   Define the sampling frequency (srate) and the number of samples that the time vector should be composed of.
   Define the time step.
   CLUE: you could use the numpy function, arange: numpy.arange(start, stop, step)
'''
srate = sfreq               # The sampling frequency.
datasize = data_dim[1]      # The number os samples in the time vector.
tstep    = 1/srate
timev    = np.arange(0, datasize*tstep, tstep)
print('The length of the time vector is: ',len(timev))
print(f'The duration of the time vector is {timev[-1]}seconds')
                

## Now plot a single channel
We will plot the Cz channel using the time vector that we just calculated above.
We use the *index()* method to get the index of the Cz channel.sfreq

In [ ]:
%matplotlib inline
''' Now we can plot the data of a single electrode over time.
    We want to plot the Cz electrode...
'''

chanidx = chan_names.index('Cz')             # Find the index of the Cz electrode.
plt.plot(timev, data1[chanidx, :])        # Plot the Cz signal
plt.xlabel('time (seconds)')

## Plot an individual channel over a defined time interval
Here we will just plot the data of the Pz channel over the 60second to 70second time interval.
This means that we need define a shorter time interval.

In [ ]:
### But we may want to visualize individual channel data for a pre-defined time interval.
"""Need to consctruct the new time vector"""
lims_sec    = np.array([60, 70])               # We will define the limits of the time interval, from 60seconds to 70seconds
lim1, lim2  = (lims_sec * srate).astype(int)   # Find the indices of the start and end of chosen time interval
chan2plot   = 'Pz'                             # The index of the channel that you want to plot, here Pz
chanindx2   = chan_names.index(chan2plot)
RawIn_sel   = data1[chanindx2, lim1:lim2]   # Extract the raw data of interest

# Now plot the time interval of data.
t = timev[lim1:lim2]                             # We define a new time vector, t, as being between lim1 and lim2
plt.plot(t,RawIn_sel)
plt.show()

In [ ]:
## Now we will plot the EEG again with one little change...

mne.viz.plot_raw(rawIn, scalings='auto', remove_dc=False)

# THE DC OFFSET

You notice that none of the electrodes appear to be visible...this is due to what we call the "**DC Offset**".
The acquisition system works on battery, so DC, and it captures ALL the frequencies including the OHz.
The OHz is the offset from zero mean. 
So we need to remove this offset; after which our signals will have a zero mean.

#### There are different ways of removing the DC Offset:

We will start by trying to remove the DC offset, by subtracting the mean activity from the activity of one channel.
Then we will plot the result.
So...
- Let's calculate the mean of a few channels.
- What do you notice about the means? How do we know that there is a DC offset?

Note also the use of the *copy()** method. We use this to make a copy of the original **RawIn** object.
When we apply a method such as, *.pick_channels*, to a raw object, we change that object. Therefore, the copy() method is very useful.

In [ ]:
## To test the effect of the DC offset, we will find the mean of a few electrodes.
"""
    Find the mean of the data from several channels.
    What can we say about the means?
"""

raw_temp   = rawIn.copy()                    # Create a copy of the raw object and name it RawIn_temp
raw_temp.pick_channels(['F3', 'Fz', 'F4'])     # We are going to compute of a subset of channels.
dataIn     = raw_temp.get_data()               # Extract the data from the RawIn object.
data_mean  = np.mean(dataIn, 1)                # We want to find the mean over the time samples, so the 2nd dimension.
Dmean      = data_mean.tolist()                # Converting the array to a list.

print('The mean for each  electrode: {} '.format(Dmean))

## Removing the DC Offset

So, if we think we need to remove the DC offset, we can do one of the following:
- Subtract the mean of each signal from each time sample of each channel.
- Carry out high-pass filtering to remove the 0Hz
- Carry out **detrending**

In the following, you can compare the effect of subtracting the mean and high-pass filtering.
We can carry out this test on the **RawIn_sel2** data, that consists of 6 channels.

In [ ]:
## Subtract the mean
# We already know the means of the 6 channels, they are stored in the Dmean list and the data_mean array.
# The channel-data of the RawIn_temp object is "dataIn"
chan_demean1 = np.subtract(dataIn[0,], Dmean[0])
chan_demean2 = np.subtract(dataIn[1,], Dmean[1])
chan_demean3 = np.subtract(dataIn[2,], Dmean[2])

%matplotlib qt
## Plot the original and demeaned channels
ax1 = plt.subplot(231)
ax1.margins(0.5)
ax1.plot(timev, dataIn[0,])

ax2 = plt.subplot(232)
ax2.margins(0.5)
ax2.plot(timev,dataIn[1,])

ax3 = plt.subplot(233)
ax3.margins(0.5)
ax3.plot(timev,dataIn[2,])

ax4 = plt.subplot(234)
ax4.margins(0.5)
ax4.plot(timev, chan_demean1)

ax5 = plt.subplot(235)
ax5.margins(0.5)
ax5.plot(timev,chan_demean2)

ax6 = plt.subplot(236)
ax6.margins(0.5)
ax6.plot(timev,chan_demean3)

# FILTERING THE EEG SIGNAL

In EEG, we generally filter to remove high frequency artifacts and low frequency drifts.
We can filter our time-domain data, our continuous EEG.
We can also filter our spatial-domain data using spatial filters.

We begin by filtering our time-domain data:
- we apply a high-pass filter to remove low frequency drifts and the DC Offset.
- we apply a low-pass filter to remove high frequency artifacts.
Here we are going to apply a *high-pass filter* only to try to remove the DC offset.

In [ ]:
## Filter the EEG Signal.
#  High-pass filter with limit of 0.1Hz. 
#  Note that we create a copy of the original rawmed1 object before filtering.

rawIn_filt = rawIn.copy().filter(0.1, None, fir_design='firwin')

## Task 2:

Compare the effect of removing the DC component by demeaning and by filtering for the following set of electrodes:
- midline electrodes
- frontal electrodes on the left hemisphere.
- frontal electrodes on the right hemisphere.

Look at the effect of high-pass filtering at 0.5Hz and 1Hz

Write your code in the cell below.

In [ ]:
## Code for Task 2 goes here.

# Exploring the EEG data using annotations





In [ ]:
## We will plot our high-pass filtered data.

rawIn_filt.plot(duration= 20, start = 60, scalings='auto', remove_dc=False)

### -------------------- PLOT TOPOGRAPHIES FOR DEFINED TIME INTERVAL -------------------------
In addition to looking at the signal as a function of time.
We can look at the spatial distribution of activity across the head (topography) for a given time interval.
We could do this to highlight activity as a specific time or verify if certain activity corresponds to an artifact.
The plot a single topography, we need to define a vector of the mean activity over a defined time interval.

- Try to find a time interval containing eye-blinks or ECG or alpha oscillation.
- Note the time interval or intervals.
- Plot the topography of the activity over this time interval.

The function to plot topography is given below.
You can use the EEG artifact, characteristics CheatSheet to help you detect these artifacts.
Note: Before we can visualise the topography, we need to define the electrode layout or **montage** that corresponds
to the current data.
Here we use the standard 10-20 montage.

<img src="figures/10-20_1.jpg" width=520 height=550 /><br />

In [ ]:
'''
    Add the montage information to the current raw object, RawIn.
    This is required if you want to plot the topography maps.
'''
montage = mne.channels.make_standard_montage('standard_1020')               # Assigning the standard 10-20 montage
mne.viz.plot_montage(mne.channels.make_standard_montage('standard_1020'))   # Visualize the montage
rawIn_filt.set_montage(montage)

In [ ]:
## When we plot the spatial distribution of activity, we pick a time interval of interest 
#  and plot the mean activity within that time interval. 

t_int = [70, 75]
timeIndx = rawIn_filt.time_as_index(t_int)
chan_range = np.arange(0,64)
dataseg1 = rawIn_filt.get_data(chan_range, timeIndx[0], timeIndx[1])
dataseg_avg = np.mean(dataseg1, 1)

fig, ax = plt.subplots(1)
mne.viz.plot_topomap(dataseg_avg, rawIn_filt.info, ch_type='eeg', axes=ax)

In [ ]:
## Now let's plot topographies over several time intervals of interest.

time_ints = np.arange(60, 64, 1)                   # Define the time intervals and time step
print(time_ints)


In [ ]:
fig, axes = plt.subplots(1, len(time_ints)-1, figsize=(15, 5))

for ind in range(len(time_ints)-1):
    curr_int =  [time_ints[ind], time_ints[ind+1]]
    print(curr_int)
    timeIndx2 = rawIn_filt.time_as_index(curr_int) 
    print(timeIndx2)
    dataseg_curr  = rawIn_filt.get_data(chan_range, timeIndx2[0], timeIndx2[1])
    datamean_curr = np.mean(dataseg_curr, 1)
    
    mne.viz.plot_topomap(datamean_curr, rawIn_filt.info, ch_type='eeg', axes=axes[ind])
    axes[ind].set_title(str(time_ints[ind])+' - '+ str(time_ints[ind+1])+'seconds', {'fontsize' : 20})
    

# Exploring the Spectral & Spectro-Temporal Characteristics of EEG

We can also study the EEG signal in terms of its spectral content. 


In [ ]:
# Plot the power spectral density over:
# - the 1Hz to 80Hz frequency band
# - a predefined time interval (60 - 80seconds)

%matplotlib widget
mne.viz.plot_raw_psd(rawIn_filt, fmin=1, fmax=100, tmin=60, tmax=80, picks='eeg', dB=True)

## Exploring the spectral content over a sub-set of channels

Plotting the average topographies for different frequency bands (theta, alpha, beta, gamma).



In [ ]:
# Plotting the topography of activity in different frequency bands.

spectra = rawIn_filt.compute_psd(method='multitaper', fmin=0.5, fmax=80, tmin=None, tmax=None, picks='eeg')   # This yields a spectra object.
psds, freqs = spectra.get_data(exclude=(), return_freqs=True)
print('The frequencies are: ', freqs)

# Plot the spectra as a function of frequency.
chan_sel = ['Fpz','Fz','FCz','Cz','CPz','Pz']
spectra.plot(picks=chan_sel)

fbands = {'Theta (4-7Hz)': (4,7), 'Alpha (8-12 Hz)': (8, 12), 'Beta (12-30 Hz)': (12, 30),
         'Gamma (30-45 Hz)': (30, 45)}

spectra.plot_topomap(bands=fbands, ch_type='eeg')

# Task 3

In the cell below, perform the following:
- Load in the dataset "sub-001_eeg_sub-001_task-think1_eeg.bdf"
- Display the following information onscreen:
                - sampling frequency
                - number of 'eeg'channels
                - the title of the 'eeg' channels
                - the number of sample points
                - the time resolution of the input data.
- In separate plots, plot the average time signal over:
                - the midline electrodes
                - RH frontal electrodes
                - LH frontal electrodes
                - RH posterior electrodes
                - LH posterior electrodes
- Plot the spectrum at three different time intervals: beginning, middle, end of data. 
                - Do you notice a difference?
                - What is the peak frequency?
- Plot the topography of alpha-band activity over the same time intervals and compare. 

- Compare the alpha-band topography over the same time intervals of the datasets, sub-001_eeg_sub-001_task-think1_eeg.bdf and sub-001_eeg_sub-001_task-med2_eeg.bdf. Do you notice any difference?

Use the code provided in the example above to help you. 
